# MediaPipe Pose を使ったリアルタイム姿勢推定

このノートブックでは、MediaPipe Poseを使用してリアルタイムで人体の姿勢推定を行います。

## MediaPipe Poseについて
- Googleが開発したオープンソースの機械学習ライブラリ
- 高精度かつ高速な姿勢推定が可能
- 33個のランドマークポイントを検出
- Webカメラやアップロード画像で利用可能

## 1. ライブラリのインストールとインポート

In [ ]:
# 必要なライブラリをインストール
!pip install mediapipe opencv-python matplotlib japanize-matplotlib -q

print("✅ ライブラリのインストールが完了しました！")

In [ ]:
import cv2
import mediapipe as mp
import japanize_matplotlib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import base64
from IPython.display import display, HTML

# MediaPipe Pose の設定
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("✅ ライブラリのインポートが完了しました！")
print(f"MediaPipe version: {mp.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 2. 画像アップロード機能

In [ ]:
# Google Colabでのファイルアップロード機能
try:
    from google.colab import files
    uploaded = files.upload()
    
    if uploaded:
        # アップロードされたファイルの最初のものを使用
        filename = list(uploaded.keys())[0]
        print(f"📁 アップロードされたファイル: {filename}")
    else:
        print("ファイルがアップロードされませんでした")
        filename = None
        
except ImportError:
    # Google Colab以外の環境の場合はサンプル画像のURLを使用
    print("🌐 サンプル画像を使用します")
    import urllib.request
    
    # サンプル画像のダウンロード
    sample_url = "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=400"
    filename = "sample_person.jpg"
    
    try:
        urllib.request.urlretrieve(sample_url, filename)
        print(f"📁 サンプル画像をダウンロードしました: {filename}")
    except:
        print("❌ サンプル画像のダウンロードに失敗しました")
        filename = None

## 3. 姿勢推定の実行

In [ ]:
def detect_pose(image_path):
    """
    画像から姿勢を検出する関数
    """
    # 画像を読み込み
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ 画像を読み込めませんでした: {image_path}")
        return None, None
    
    # BGR から RGB に変換
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # MediaPipe Pose で姿勢推定
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5
    ) as pose:
        
        results = pose.process(image_rgb)
        
        # 結果を描画
        annotated_image = image_rgb.copy()
        
        if results.pose_landmarks:
            landmark_style = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=5, circle_radius=10)
            connection_style = mp_drawing.DrawingSpec(color=(0, 255,), thickness=3)

            # ランドマークとスケルトンを描画
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=landmark_style,
                connection_drawing_spec=connection_style
            )
            
            print("✅ 姿勢が検出されました！")
            print(f"検出されたランドマーク数: {len(results.pose_landmarks.landmark)}")
        else:
            print("⚠️ 姿勢が検出されませんでした")
    
    return image_rgb, annotated_image

# 姿勢推定を実行
if filename:
    original_image, pose_image = detect_pose(filename)
    
    if original_image is not None and pose_image is not None:
        # 結果を表示
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # 元の画像
        axes[0].imshow(original_image)
        axes[0].set_title('元の画像')
        axes[0].axis('off')
        
        # 姿勢推定結果
        axes[1].imshow(pose_image)
        axes[1].set_title('姿勢推定結果')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
else:
    print("❌ 処理する画像がありません")

## 4. ランドマーク情報の表示

In [ ]:
def show_landmark_info(image_path):
    """
    ランドマーク情報を詳細に表示する関数
    """
    image = cv2.imread(image_path)
    if image is None:
        return
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        min_detection_confidence=0.5
    ) as pose:
        
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            # 主要なランドマークの名前
            landmark_names = [
                "鼻", "左目（内側）", "左目", "左目（外側）", "右目（内側）",
                "右目", "右目（外側）", "左耳", "右耳", "左口角",
                "右口角", "左肩", "右肩", "左肘", "右肘",
                "左手首", "右手首", "左小指", "右小指", "左人差し指",
                "右人差し指", "左親指", "右親指", "左腰", "右腰",
                "左膝", "右膝", "左足首", "右足首", "左踵",
                "右踵", "左足先", "右足先"
            ]
            
            print("🔍 検出されたランドマーク情報:")
            print("=" * 50)
            
            for i, (landmark, name) in enumerate(zip(results.pose_landmarks.landmark, landmark_names)):
                print(f"{i:2d}. {name:12}: x={landmark.x:.3f}, y={landmark.y:.3f}, z={landmark.z:.3f}, visibility={landmark.visibility:.3f}")
                
                # 最初の10個だけ表示（表示を簡潔にするため）
                if i >= 9:
                    print("   ... (残りのランドマークは省略)")
                    break
        else:
            print("❌ ランドマークが検出されませんでした")

# ランドマーク情報を表示
if filename:
    show_landmark_info(filename)

## 5. まとめ

### 🎯 このノートブックで学んだこと

1. **MediaPipe Pose の基本的な使い方**
   - 簡単なセットアップと設定
   - 画像からの姿勢推定

2. **姿勢推定の結果**
   - 33個のランドマークポイントの検出
   - スケルトンの可視化
   - 座標情報の取得

3. **応用可能性**
   - フィットネスアプリ
   - 動作解析
   - ゲーム制御
   - 医療・リハビリ支援

### 💡 次のステップ

- Webカメラを使ったリアルタイム姿勢推定
- 動画ファイルの姿勢推定
- 特定の動作やポーズの判定
- 複数人の姿勢推定

---

**🎉 MediaPipe Pose を使った姿勢推定が完了しました！**

{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "title-cell",
   "metadata": {},
   "source": [
    "# MediaPipe Pose を使ったリアルタイム姿勢推定\n",
    "\n",
    "このノートブックでは、MediaPipe Poseを使用してリアルタイムで人体の姿勢推定を行います。\n",
    "\n",
    "## MediaPipe Poseについて\n",
    "- Googleが開発したオープンソースの機械学習ライブラリ\n",
    "- 高精度かつ高速な姿勢推定が可能\n",
    "- 33個のランドマークポイントを検出\n",
    "- Webカメラやアップロード画像で利用可能"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "section1-title",
   "metadata": {},
   "source": [
    "## 1. ライブラリのインストールとインポート"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "install-libraries",
   "metadata": {},
   "outputs": [],
   "source": [
    "# 必要なライブラリをインストール\n",
    "!pip install mediapipe opencv-python matplotlib\n",
    "\n",
    "print(\"✅ ライブラリのインストールが完了しました！\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "import-libraries",
   "metadata": {},
   "outputs": [],
   "source": [
    "import cv2\n",
    "import mediapipe as mp\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from PIL import Image\n",
    "import io\n",
    "import base64\n",
    "from IPython.display import display, HTML\n",
    "\n",
    "# MediaPipe Pose の設定\n",
    "mp_pose = mp.solutions.pose\n",
    "mp_drawing = mp.solutions.drawing_utils\n",
    "mp_drawing_styles = mp.solutions.drawing_styles\n",
    "\n",
    "print(\"✅ ライブラリのインポートが完了しました！\")\n",
    "print(f\"MediaPipe version: {mp.__version__}\")\n",
    "print(f\"OpenCV version: {cv2.__version__}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "section2-title",
   "metadata": {},
   "source": [
    "## 2. 画像アップロード機能"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "upload-image",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Google Colabでのファイルアップロード機能\n",
    "try:\n",
    "    from google.colab import files\n",
    "    uploaded = files.upload()\n",
    "    \n",
    "    if uploaded:\n",
    "        # アップロードされたファイルの最初のものを使用\n",
    "        filename = list(uploaded.keys())[0]\n",
    "        print(f\"📁 アップロードされたファイル: {filename}\")\n",
    "    else:\n",
    "        print(\"ファイルがアップロードされませんでした\")\n",
    "        filename = None\n",
    "        \n",
    "except ImportError:\n",
    "    # Google Colab以外の環境の場合はサンプル画像のURLを使用\n",
    "    print(\"🌐 サンプル画像を使用します\")\n",
    "    import urllib.request\n",
    "    \n",
    "    # サンプル画像のダウンロード\n",
    "    sample_url = \"https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=400\"\n",
    "    filename = \"sample_person.jpg\"\n",
    "    \n",
    "    try:\n",
    "        urllib.request.urlretrieve(sample_url, filename)\n",
    "        print(f\"📁 サンプル画像をダウンロードしました: {filename}\")\n",
    "    except:\n",
    "        print(\"❌ サンプル画像のダウンロードに失敗しました\")\n",
    "        filename = None"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "section3-title",
   "metadata": {},
   "source": [
    "## 3. 姿勢推定の実行"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "pose-detection",
   "metadata": {},
   "outputs": [],
   "source": [
    "def detect_pose(image_path):\n",
    "    \"\"\"\n",
    "    画像から姿勢を検出する関数\n",
    "    \"\"\"\n",
    "    # 画像を読み込み\n",
    "    image = cv2.imread(image_path)\n",
    "    if image is None:\n",
    "        print(f\"❌ 画像を読み込めませんでした: {image_path}\")\n",
    "        return None, None\n",
    "    \n",
    "    # BGR から RGB に変換\n",
    "    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n",
    "    \n",
    "    # MediaPipe Pose で姿勢推定\n",
    "    with mp_pose.Pose(\n",
    "        static_image_mode=True,\n",
    "        model_complexity=2,\n",
    "        enable_segmentation=True,\n",
    "        min_detection_confidence=0.5\n",
    "    ) as pose:\n",
    "        \n",
    "        results = pose.process(image_rgb)\n",
    "        \n",
    "        # 結果を描画\n",
    "        annotated_image = image_rgb.copy()\n",
    "        \n",
    "        if results.pose_landmarks:\n",
    "            # ランドマークとスケルトンを描画\n",
    "            mp_drawing.draw_landmarks(\n",
    "                annotated_image,\n",
    "                results.pose_landmarks,\n",
    "                mp_pose.POSE_CONNECTIONS,\n",
    "                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()\n",
    "            )\n",
    "            \n",
    "            print(\"✅ 姿勢が検出されました！\")\n",
    "            print(f\"検出されたランドマーク数: {len(results.pose_landmarks.landmark)}\")\n",
    "        else:\n",
    "            print(\"⚠️ 姿勢が検出されませんでした\")\n",
    "    \n",
    "    return image_rgb, annotated_image\n",
    "\n",
    "# 姿勢推定を実行\n",
    "if filename:\n",
    "    original_image, pose_image = detect_pose(filename)\n",
    "    \n",
    "    if original_image is not None and pose_image is not None:\n",
    "        # 結果を表示\n",
    "        fig, axes = plt.subplots(1, 2, figsize=(12, 6))\n",
    "        \n",
    "        # 元の画像\n",
    "        axes[0].imshow(original_image)\n",
    "        axes[0].set_title('元の画像')\n",
    "        axes[0].axis('off')\n",
    "        \n",
    "        # 姿勢推定結果\n",
    "        axes[1].imshow(pose_image)\n",
    "        axes[1].set_title('姿勢推定結果')\n",
    "        axes[1].axis('off')\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "else:\n",
    "    print(\"❌ 処理する画像がありません\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "section4-title",
   "metadata": {},
   "source": [
    "## 4. ランドマーク情報の表示"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "landmark-info",
   "metadata": {},
   "outputs": [],
   "source": [
    "def show_landmark_info(image_path):\n",
    "    \"\"\"\n",
    "    ランドマーク情報を詳細に表示する関数\n",
    "    \"\"\"\n",
    "    image = cv2.imread(image_path)\n",
    "    if image is None:\n",
    "        return\n",
    "    \n",
    "    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n",
    "    \n",
    "    with mp_pose.Pose(\n",
    "        static_image_mode=True,\n",
    "        model_complexity=2,\n",
    "        min_detection_confidence=0.5\n",
    "    ) as pose:\n",
    "        \n",
    "        results = pose.process(image_rgb)\n",
    "        \n",
    "        if results.pose_landmarks:\n",
    "            # 主要なランドマークの名前\n",
    "            landmark_names = [\n",
    "                \"鼻\", \"左目（内側）\", \"左目\", \"左目（外側）\", \"右目（内側）\",\n",
    "                \"右目\", \"右目（外側）\", \"左耳\", \"右耳\", \"左口角\",\n",
    "                \"右口角\", \"左肩\", \"右肩\", \"左肘\", \"右肘\",\n",
    "                \"左手首\", \"右手首\", \"左小指\", \"右小指\", \"左人差し指\",\n",
    "                \"右人差し指\", \"左親指\", \"右親指\", \"左腰\", \"右腰\",\n",
    "                \"左膝\", \"右膝\", \"左足首\", \"右足首\", \"左踵\",\n",
    "                \"右踵\", \"左足先\", \"右足先\"\n",
    "            ]\n",
    "            \n",
    "            print(\"🔍 検出されたランドマーク情報:\")\n",
    "            print(\"=" * 50)\n",
    "            \n",
    "            for i, (landmark, name) in enumerate(zip(results.pose_landmarks.landmark, landmark_names)):\n",
    "                print(f\"{i:2d}. {name:12}: x={landmark.x:.3f}, y={landmark.y:.3f}, z={landmark.z:.3f}, visibility={landmark.visibility:.3f}\")\n",
    "                \n",
    "                # 最初の10個だけ表示（表示を簡潔にするため）\n",
    "                if i >= 9:\n",
    "                    print(\"   ... (残りのランドマークは省略)\")\n",
    "                    break\n",
    "        else:\n",
    "            print(\"❌ ランドマークが検出されませんでした\")\n",
    "\n",
    "# ランドマーク情報を表示\n",
    "if filename:\n",
    "    show_landmark_info(filename)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "summary",
   "metadata": {},
   "source": [
    "## 5. まとめ\n",
    "\n",
    "### 🎯 このノートブックで学んだこと\n",
    "\n",
    "1. **MediaPipe Pose の基本的な使い方**\n",
    "   - 簡単なセットアップと設定\n",
    "   - 画像からの姿勢推定\n",
    "\n",
    "2. **姿勢推定の結果**\n",
    "   - 33個のランドマークポイントの検出\n",
    "   - スケルトンの可視化\n",
    "   - 座標情報の取得\n",
    "\n",
    "3. **応用可能性**\n",
    "   - フィットネスアプリ\n",
    "   - 動作解析\n",
    "   - ゲーム制御\n",
    "   - 医療・リハビリ支援\n",
    "\n",
    "### 💡 次のステップ\n",
    "\n",
    "- Webカメラを使ったリアルタイム姿勢推定\n",
    "- 動画ファイルの姿勢推定\n",
    "- 特定の動作やポーズの判定\n",
    "- 複数人の姿勢推定\n",
    "\n",
    "---\n",
    "\n",
    "**🎉 MediaPipe Pose を使った姿勢推定が完了しました！**"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

## 1. ライブラリのインストールとインポート

In [ ]:
# 必要なライブラリをインストール
!pip install mediapipe opencv-python matplotlib

print("✅ ライブラリのインストールが完了しました！")

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import base64
from IPython.display import display, HTML

# MediaPipe Pose の設定
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("✅ ライブラリのインポートが完了しました！")
print(f"MediaPipe version: {mp.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 2. 画像アップロード機能

In [ ]:
# Google Colabでのファイルアップロード機能
try:
    from google.colab import files
    uploaded = files.upload()
    
    if uploaded:
        # アップロードされたファイルの最初のものを使用
        filename = list(uploaded.keys())[0]
        print(f"📁 アップロードされたファイル: {filename}")
    else:
        print("ファイルがアップロードされませんでした")
        filename = None
        
except ImportError:
    # Google Colab以外の環境の場合はサンプル画像のURLを使用
    print("🌐 サンプル画像を使用します")
    import urllib.request
    
    # サンプル画像のダウンロード
    sample_url = "https://images.unsplash.com/photo-1571019613454-1cb2f99b2d8b?w=400"
    filename = "sample_person.jpg"
    
    try:
        urllib.request.urlretrieve(sample_url, filename)
        print(f"📁 サンプル画像をダウンロードしました: {filename}")
    except:
        print("❌ サンプル画像のダウンロードに失敗しました")
        filename = None

## 3. 姿勢推定の実行

In [ ]:
def detect_pose(image_path):
    """
    画像から姿勢を検出する関数
    """
    # 画像を読み込み
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ 画像を読み込めませんでした: {image_path}")
        return None, None
    
    # BGR から RGB に変換
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # MediaPipe Pose で姿勢推定
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5
    ) as pose:
        
        results = pose.process(image_rgb)
        
        # 結果を描画
        annotated_image = image_rgb.copy()
        
        if results.pose_landmarks:
            # ランドマークとスケルトンを描画
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )
            
            print("✅ 姿勢が検出されました！")
            print(f"検出されたランドマーク数: {len(results.pose_landmarks.landmark)}")
        else:
            print("⚠️ 姿勢が検出されませんでした")
    
    return image_rgb, annotated_image

# 姿勢推定を実行
if filename:
    original_image, pose_image = detect_pose(filename)
    
    if original_image is not None and pose_image is not None:
        # 結果を表示
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # 元の画像
        axes[0].imshow(original_image)
        axes[0].set_title('元の画像')
        axes[0].axis('off')
        
        # 姿勢推定結果
        axes[1].imshow(pose_image)
        axes[1].set_title('姿勢推定結果')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
else:
    print("❌ 処理する画像がありません")

## 4. ランドマーク情報の表示

In [ ]:
def show_landmark_info(image_path):
    """
    ランドマーク情報を詳細に表示する関数
    """
    image = cv2.imread(image_path)
    if image is None:
        return
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        min_detection_confidence=0.5
    ) as pose:
        
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            # 主要なランドマークの名前
            landmark_names = [
                "鼻", "左目（内側）", "左目", "左目（外側）", "右目（内側）",
                "右目", "右目（外側）", "左耳", "右耳", "左口角",
                "右口角", "左肩", "右肩", "左肘", "右肘",
                "左手首", "右手首", "左小指", "右小指", "左人差し指",
                "右人差し指", "左親指", "右親指", "左腰", "右腰",
                "左膝", "右膝", "左足首", "右足首", "左踵",
                "右踵", "左足先", "右足先"
            ]
            
            print("🔍 検出されたランドマーク情報:")
            print("=" * 50)
            
            for i, (landmark, name) in enumerate(zip(results.pose_landmarks.landmark, landmark_names)):
                print(f"{i:2d}. {name:12}: x={landmark.x:.3f}, y={landmark.y:.3f}, z={landmark.z:.3f}, visibility={landmark.visibility:.3f}")
                
                # 最初の10個だけ表示（表示を簡潔にするため）
                if i >= 9:
                    print("   ... (残りのランドマークは省略)")
                    break
        else:
            print("❌ ランドマークが検出されませんでした")

# ランドマーク情報を表示
if filename:
    show_landmark_info(filename)

## 5. まとめ

### 🎯 このノートブックで学んだこと

1. **MediaPipe Pose の基本的な使い方**
   - 簡単なセットアップと設定
   - 画像からの姿勢推定

2. **姿勢推定の結果**
   - 33個のランドマークポイントの検出
   - スケルトンの可視化
   - 座標情報の取得

3. **応用可能性**
   - フィットネスアプリ
   - 動作解析
   - ゲーム制御
   - 医療・リハビリ支援

### 💡 次のステップ

- Webカメラを使ったリアルタイム姿勢推定
- 動画ファイルの姿勢推定
- 特定の動作やポーズの判定
- 複数人の姿勢推定

---

**🎉 MediaPipe Pose を使った姿勢推定が完了しました！**